# HW 2 - Naive Bayes in Hadoop MR
__`MIDS w261: Machine Learning at Scale | UC Berkeley School of Information | Fall 2018`__

In the live sessions for week 2 and week 3 you got some practice designing and debugging Hadoop Streaming jobs. In this homework we'll use Hadoop MapReduce to implement your first parallelized machine learning algorithm: Naive Bayes. As you develop your implementation you'll test it on a small dataset that matches the 'Chinese Example' in the _Manning, Raghavan and Shutze_ reading for Week 2. For the main task in this assignment you'll be working with a small subset of the Enron Spam/Ham Corpus. By the end of this assignment you should be able to:
* __... describe__ the Naive Bayes algorithm including both training and inference.
* __... perform__ EDA on a corpus using Hadoop MR.
* __... implement__ parallelized Naive Bayes.
* __... constrast__ partial, unordered and total order sort and their implementations in Hadoop Streaming.
* __... explain__ how smoothing affects the bias and variance of a Multinomial Naive Bayes model.

As always, your work will be graded both on the correctness of your output and on the clarity and design of your code. __Please refer to the `README` for homework submission instructions.__ 

## Notebook Setup
Before starting, run the following cells to confirm your setup.

In [7]:
# imports
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [27]:
# global vars (paths) - ADJUST AS NEEDED
JAR_FILE = "/usr/lib/hadoop-mapreduce/hadoop-streaming.jar"
HDFS_DIR = "/user/root/HW2"
HOME_DIR = "/media/notebooks/Assignments/HW2" # FILL IN HERE eg. /media/notebooks/Assignments/HW2

In [28]:
# save path for use in Hadoop jobs (-cmdenv PATH={PATH})
from os import environ
PATH  = environ['PATH']

In [29]:
# data path
ENRON = "./data/enronemail_1h.txt"

# Question 1: Hadoop MapReduce Key Takeaways.  

This assignment will be the only one in which you use Hadoop Streaming to implement a distributed algorithm. The key reason we continue to teach Hadoop streaming is because of the way it forces the programmer to think carefully about what is happening under the hood when you parallelize a calculation. This question will briefly highlight some of the most important concepts that you need to understand about Hadoop Streaming and MapReduce before we move on to Spark next week.   

### Q1 Tasks:

* __a) short response:__ What "programming paradigm" is Hadoop MapReduce based on? What are the main ideas of this programming paradigm and how does MapReduce exemplify these ideas?

* __b) short response:__ What is the Hadoop Shuffle? When does it happen? Why is it potentially costly? Describe one specific thing we can we do to mitigate the cost associated with this stage of our Hadoop Streaming jobs.

* __c) short response:__ In Hadoop Streaming why do the input and output record format of a combiner script have to be the same? [__`HINT`__ _what level of combining does the framework guarantee? what is the relationship between the record format your mapper emits and the format your reducer expects to receive?_]

* __d) short response:__ To what extent can you control the level of parallelization of your Hadoop Streaming jobs? Please be specific.

* __e) short response:__ What change in the kind of computing resources available prompted the creation of parallel computation frameworks like Hadoop? 

### Q1 Student Answers:

> __a)__ Hadoop MapReduce follows "functional programming" paradigm. In functional programming, we have higher order functions that can take other low order functions as arguments. In MapReduce, we have mappers and reducers that are higher order functions. A mapper takes as an argument a function, say 'f' and applies it to all the elements of the list it gets as input. Since the function 'f' does not depend on any thing else other than its input one can easily parallelize this by running multiple higher order mappers. Similarly a reducer applies a  function, say 'g', to all the elements it gets as inputs in order to aggregate and provide the final result. 

> __b)__ In order to avoid the complexity of a reducer in aggregating the results, Hadoop MapReduce framework gaurantees a sorted list of data to reducers. The process of taking output from each mapper and sorting them before handing it to reducers is called shuffling. In order to sort the data, Hadoop needs to move the data from the mappers which can be an expensive disk I/O and Networking operation. One optimization that can be done is to use a combiner which is a method of local aggregation. For eg., in a word-count problem, a combiner will aggregate each key count locally per mapper, there by the amount of intermediate key-value pairs is at the most equal to the number of unique keys.   

> __c)__ Hadoop framework considers combiner as an optimization function. As such, there is no guarantee on how many times a combiner is applied or if it is applied at all. In other words, a combiner is treated as a bump-in-a-wire between mapper and reducer. So the input/output format of combiner must always match the output of the mapper which is in the form of key-value pair. The reducer code must not change with the presence/absence of combiner and expects the same key-value pair format.  

> __d)__ A user can control the below:
* Specify number of mappers or reducers. Right number of mappers is <100.
* Can specify initialization code at the beginning of the mapper
* Can specify termination code at the end of map or reduce task
* Control the sort order of the intermediate keys (produced by the mapper/combiner) 
* Control the partitioning of the key space, and hence the set of keys processed by a reducer.

> __e)__ Moore's law started to decline where the processing speeds no longer double every year. So, instead of developing a custom scale-up solution which is expensive, people started to look for scale-out designs. Evolution of commoditized servers that are built on standard components for processing, storage and networking helped scale-out further. Volume drove the cost of cheaper servers down. Applications are being developed to take advantage of scale-out model for parallelism and resiliency.   

# Question 2: MapReduce Design Patterns.  

In the last two live sessions and in your readings from Lin & Dyer you encountered a number of techniques for manipulating the logistics of a MapReduce implementation to ensure that the right information is available at the right time and location. In this question we'll review a few of the key techniques you learned.   

### Q2 Tasks:

* __a) short response:__ What are counters (in the context of Hadoop Streaming)? How are they useful? What kinds of counters does Hadoop provide for you? How do you create your own custom counter?

* __b) short response:__ What are composite keys? How are they useful? How are they related to the idea of custom partitioning?

* __c) short response:__ What is the order inversion pattern? What problem does it help solve? How do we implement it? 

### Q2 Student Answers:

> __a)__ In Hadoop framework, counters are used to maintain statistics of common parameters used in mappers or reducers. For example, Hadoop framework provides built-in counters for number of mappers or reducers, number of records in and out of mapper or reducer, CPU and memory utilized etc. We can also create custom counters by declaring a global variable and updating it based on some custom conditions. Hadoop framework aggregates these global counters across mappers and reducers and make sure it is available at the end of the job. This is very useful for debugging purposes.

> __b)__ A composite key consists of multiple keys that are joined together. We can use composite keys for secondary sorting, whenever there is a need to do a sort based on primary key and a subsequent sort based on the value. In custom partitioning (used in Q8 below), we create a custom partition key and nest it along with our key/value pair. We then use the custom key to force partitioning based on user logic. A secondary sort on the second key would then allow the reducer logic to work effectively. Using custom partitioning and composite key concept, we can push certain keys (like TOTAL WORD COUNT) to be available to all the reducers.    

> __c)__ An order inversion is the mechanism where the data going into the reducer is sorted in such a way that aggregate count is calculated first. For example, in Naive Bayes word counting problem, we need to find the "total words per class" first, before we can calculate conditional probabilities. With order inversion, we emit a special character key representing the aggregate value (say '*'). This special character key is sorted and calculated first before the other word counts are done.


# Question 3: Understanding Total Order Sort

The key challenge in distributed computing is to break a problem into a set of sub-problems that can be performed without communicating with each other. Ideally, we should be able to define an arbirtary number of splits and still get the right result, but that is not always possible. Parallelization becomes particularly challenging when we need to make comparisons between records, for example when sorting. Total Order Sort allows us to order large datasets in a way that enables efficient retrieval of results. Before beginning this assignment, make sure you have read and understand the [Total Order Sort Notebook](https://github.com/UCB-w261/main/tree/master/HelpfulResources/TotalSortGuide/_total-sort-guide-spark2.01-JAN27-2017.ipynb). You can skip the first two MRJob sections, but the rest of section III and all of section IV are **very** important (and apply to Hadoop Streaming) so make sure to read them closely. Feel free to read the Spark sections as well but you won't be responsible for that material until later in the course. To verify your understanding, answer the following questions.

### Q3 Tasks:

* __a) short response:__ What is the difference between a partial sort, an unordered total sort, and a total order sort? From the programmer's perspective, what does total order sort allow us to do that we can't with unordered total? Why is this important with large datasets?

* __b) short response:__ Which phase of a MapReduce job is leveraged to implement Total Order Sort? Which default behaviors must be changed. Why must they be changed?

* __c) short response:__ Describe in words how to configure a Hadoop Streaming job for the custom sorting and partitioning that is required for Total Order Sort.  

* __d) short response:__ Explain why we need to use an inverse hash code function.

* __e) short response:__ Where does this function need to be located so that a Total Order Sort can be performed?

### Q3 Student Answers:

> __a)__ 

* Partial sort is the default behavior of Hadoop mapreduce framework. In partial sort, the data is sorted within each partition. However, there is no sorting across the partitions. For example, partition 1 may have key IDs 10,8,5,2, partition 2 may have key IDs 11,7,4,1 and so on. Within the partition, the information is sorted, however across the partitions it is not.
* In an unordered total sort we use custom partition keys to make sure that we have "key ranges" landing on different partitions. However, there is no guarantee that first partition has the highest key ID followed by the second partition and so on. For example, partition 1 may have key IDs, 5,4,2,1, partition 2 may have 21,18,17,15 and partition 3 may have 11,8,7,5.
* In an ordered total sort, we use custom partition keys along with Hadoop hashing algorithm to make sure that the keys are sorted across the partitions and the partitions themselves are sorted. In the above example, with ordered total sort we will have partition 1 with key IDs, 21,18,17,15, partition 2 will have 11,8,7,5 and partition 3 will have 5,4,2,1.

Ordered total sort is useful when we want the results to be determenistic, for example the specific reducer where certain keys land.

> __b)__ We need to change the mapper to introduce the logic of inserting custom partition keys. Then the shuffle logic has to sort based on partition key and perform secondary sort on the original key. Since Hadoop's default behavior is to perform partial sort (as explained above), we are using the trick of custom partition key to force Hadoop to do shuffling/sorting differently.  

> __c)__
* We need to configure the Hadoop to use keyfields instead of default
* We need to specify number of key fields and the separator
* We need to specify keypartitioner options to sort based on custom partition key
* We need to specify keycomparator option to perform secondary sort on the primary key

> __d)__   Inverse hash code is needed to perform ordered total sort. Inverse hash code takes the patition key and returns the partition index Hadoop will return for that key (The algorithm mimics Hadoop's hashing algorithm). Using this method, we can make sure that the data can be sent to the right partition index. 

> __e)__ It should be located in the mapper. Mapper appends the custom parition key so that the right partitioning is performed. 

# About the Data
For the main task in this portion of the homework you will train a classifier to determine whether an email represents spam or not. You will train your Naive Bayes model on a 100 record subset of the Enron Spam/Ham corpus available in the HW2 data directory (__`HW2/data/enronemail_1h.txt`__).

__Source:__   
The original data included about 93,000 emails which were made public after the company's collapse. There have been a number raw and preprocessed versions of this corpus (including those available [here](http://www.aueb.gr/users/ion/data/enron-spam/index.html) and [here](http://www.aueb.gr/users/ion/publications.html)). The subset we will use is limited to emails from 6 Enron employees and a number of spam sources. It is part of [this data set](http://www.aueb.gr/users/ion/data/enron-spam/) which was created by researchers working on personlized Bayesian spam filters. Their original publication is [available here](http://www.aueb.gr/users/ion/docs/ceas2006_paper.pdf). __`IMPORTANT!`__ _For this homework please limit your analysis to the 100 email subset which we provide. No need to download or run your analysis on any of the original datasets, those links are merely provided as context._

__Preprocessing:__  
For their work, Metsis et al. (the authors) appeared to have pre-processed the data, not only collapsing all text to lower-case, but additionally separating "words" by spaces, where "words" unfortunately include punctuation. As a concrete example, the sentence:  
>  `Hey Jon, I hope you don't get lost out there this weekend!`  

... would have been reduced by Metsis et al. to the form:  
> `hey jon , i hope you don ' t get lost out there this weekend !` 

... so we have reverted the data back toward its original state, removing spaces so that our sample sentence would now look like:
> `hey jon, i hope you don't get lost out there this weekend!`  

Thus we have at least preserved contractions and other higher-order lexical forms. However, one must be aware that this reversion is not complete, and that some object (specifically web sites) will be ill-formatted, and that all text is still lower-cased.


__Format:__   
All messages are collated to a tab-delimited format:  

>    `ID \t SPAM \t SUBJECT \t CONTENT \n`  

where:  
>    `ID = string; unique message identifier`  
    `SPAM = binary; with 1 indicating a spam message`  
    `SUBJECT = string; title of the message`  
    `CONTENT = string; content of the message`   
    
Note that either of `SUBJECT` or `CONTENT` may be "NA", and that all tab (\t) and newline (\n) characters have been removed from both of the `SUBJECT` and `CONTENT` columns.  

In [13]:
# take a look at the first 100 characters of the first 5 records (RUN THIS CELL AS IS)
!head -n 5 {ENRON} | cut -c-100

0001.1999-12-10.farmer	0	 christmas tree farm pictures	NA
0001.1999-12-10.kaminski	0	 re: rankings	 thank you.
0001.2000-01-17.beck	0	 leadership development pilot	" sally:  what timing, ask and you shall receiv
0001.2000-06-06.lokay	0	" key dates and impact of upcoming sap implementation over the next few week
0001.2001-02-07.kitchen	0	 key hr issues going forward	 a) year end reviews-report needs generating 


In [14]:
# see how many messages/lines are in the file 
#(this number may be off by 1 if the last line doesn't end with a newline)
!wc -l {ENRON}

100 ./data/enronemail_1h.txt


In [15]:
# make the HDFS directory if it doesn't already exist
!hdfs dfs -mkdir {HDFS_DIR}

In [16]:
# load the data into HDFS (RUN THIS CELL AS IS)
!hdfs dfs -copyFromLocal {ENRON} {HDFS_DIR}/enron.txt

# Question 4:  Enron Ham/Spam EDA.
Before building our classifier, lets get aquainted with our data. In particular, we're interested in which words occur more in spam emails than in real emails. In this question you'll implement two Hadoop MapReduce jobs to count and sort word occurrences by document class. You'll also learn about two new Hadoop streaming parameters that will allow you to control how the records output of your mappers are partitioned for reducing on separate nodes. 

__`IMPORTANT NOTE:`__ For this question and all subsequent items, you should include both the subject and the body of the email in your analysis (i.e. concatetate them to get the 'text' of the document).

### Q4 Tasks:
* __a) code:__ Complete the missing components of the code in __`EnronEDA/mapper.py`__ and __`EnronEDA/reducer.py`__ to create a Hadoop MapReduce job that counts how many times each word in the corpus occurs in an email for each class. Pay close attention to the data format specified in the docstrings of these scripts _-- there are a number of ways to accomplish this task, we've chosen this format to help illustrate a technique in `part e`_. Run the provided unit tests to confirm that your code works as expected then run the provided Hadoop streaming command to apply your analysis to the Enron data.


* __b) code + short response:__ How many times does the word "__assistance__" occur in each class? (`HINT:` Use a `grep` command to read from the results file you generated in '`a`' and then report the answer in the space provided.)


* __c) short response:__ Would it have been possible to add some sorting parameters to the Hadoop streaming command that would cause our `part a` results to be sorted by count? Briefly explain why or why not.


* __d) code + short response:__ Write a second Hadoop MapReduce job to sort the output of `part a` first by class and then by count. Run your job and save the results to a local file using the provide code. Then describe in words how you would go about printing the top 10 words in each class given this sorted output. (`HINT 1:` _remember that you can simply pass the `part a` output directory to the input field of this job; `HINT 2:` since this task is just reodering the records from `part a` we don't need to write a mapper or reducer, just use `/bin/cat` for both_)


* __ e) code:__ A more efficient alternative to '`grep`-ing' for the top 10 words in each class would be to use the Hadoop framework to separate records from each class into its own partition so that we can just read the top lines in each. Edit your job from ` part d` to specify 2 reduce tasks and to tell Hadoop to partition based on the second field (which indicates spam/ham in our data). Your code should maintain the secondary sort -- that is each partition should list words from most to least frequent.

### Q4 Student Answers:
> __b)__ The word "assistance" appears 8 times in spam and 2 times in ham 

> __c)__ No, not for the current task. The reducer logic expects the word to be sorted. So if we sort by count, our reducer logic will break. We can take the output of the reducer and pass it to another Hadoop streaming task to sort (task d and e) 

> __d)__ Since the sorted data is already partitioned by class, we can just use "head" to display the top 10 lines of each file. Code in the below section shows the example   

In [17]:
# part a - do your work in the provided scripts then RUN THIS CELL AS IS
!chmod a+x EnronEDA/mapper.py
!chmod a+x EnronEDA/reducer.py

In [18]:
# part a - unit test EnronEDA/mapper.py (RUN THIS CELL AS IS)
!echo -e "d1	1	title	body\nd2	0	title	body" | EnronEDA/mapper.py

title	1	1
body	1	1
title	0	1
body	0	1


In [33]:
# part a - unit test EnronEDA/reducer.py (RUN THIS CELL AS IS)
!echo -e "one	1	1\none	0	1\none	0	1\ntwo	0	1" | EnronEDA/reducer.py

one	1	1
one	0	2
two	1	1
two	0	0


In [34]:
# part a - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [35]:
# part a - Hadoop streaming job (RUN THIS CELL AS IS)
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob5811860102668425292.jar tmpDir=null
19/01/22 02:41:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/22 02:41:07 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/22 02:41:08 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/22 02:41:08 INFO mapreduce.JobSubmitter: number of splits:2
19/01/22 02:41:08 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1547919087985_0003
19/01/22 02:41:08 INFO impl.YarnClientImpl: Submitted application application_1547919087985_0003
19/01/22 02:41:08 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1547919087985_0003/
19/01/22 02:41:08 INFO mapreduce.Job: Running job: job_1547919087985_0003
19/01/22 02:41:14 INFO mapreduce.Job: Job job_1547919087985_0003 running in uber mode : false
19/01/22 02:41:14 INFO mapreduce.Job:  map 0% reduce 0%
19/01

In [36]:
# part a - retrieve results from HDFS & copy them into a local file (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/eda-output/part-0000* > EnronEDA/results.txt

In [37]:
# part b - write your grep command here
!cat EnronEDA/results.txt | grep "assistance"

assistance	1	8
assistance	0	2


In [53]:
# part d/e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-sort-output

Deleted /user/root/HW2/eda-sort-output


In [54]:
# part d/e - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -D stream.num.map.output.key.fields=3 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2nr -k3,3nr" \
  -D mapreduce.partition.keypartitioner.options="-k2,2" \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper /bin/cat \
  -reducer /bin/cat \
  -input {HDFS_DIR}/eda-output \
  -output {HDFS_DIR}/eda-sort-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob3039619113824019491.jar tmpDir=null
19/01/22 16:54:35 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/22 16:54:36 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/22 16:54:36 INFO mapred.FileInputFormat: Total input paths to process : 2
19/01/22 16:54:36 INFO mapreduce.JobSubmitter: number of splits:2
19/01/22 16:54:36 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1547919087985_0008
19/01/22 16:54:37 INFO impl.YarnClientImpl: Submitted application application_1547919087985_0008
19/01/22 16:54:37 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1547919087985_0008/
19/01/22 16:54:37 INFO mapreduce.Job: Running job: job_1547919087985_0008
19/01/22 16:54:43 INFO mapreduce.Job: Job job_1547919087985_0008 running in uber mode : false
19/01/22 16:54:43 INFO mapreduce.Job:  map 0% reduce 0%
19/01

In [56]:
# part d : List the files
!hdfs dfs -ls {HDFS_DIR}/eda-sort-output/

Found 3 items
-rw-r--r--   1 root supergroup          0 2019-01-22 16:54 /user/root/HW2/eda-sort-output/_SUCCESS
-rw-r--r--   1 root supergroup      64598 2019-01-22 16:54 /user/root/HW2/eda-sort-output/part-00000
-rw-r--r--   1 root supergroup      64698 2019-01-22 16:54 /user/root/HW2/eda-sort-output/part-00001


In [57]:
# part d continuation. Save it in local file
!hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-00000 > EnronEDA/sorted_results_0.txt
!hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-00001 > EnronEDA/sorted_results_1.txt


In [60]:
# part d continuation. List top 10 results for ham
!head -10 EnronEDA/sorted_results_0.txt

the	0	549	
to	0	398	
ect	0	381	
and	0	277	
of	0	230	
hou	0	205	
a	0	195	
in	0	181	
for	0	169	
on	0	135	


In [61]:
# part d continuation. List top 10 results for spam
!head -10 EnronEDA/sorted_results_1.txt

the	1	698	
to	1	566	
and	1	393	
your	1	357	
a	1	348	
you	1	345	
of	1	336	
in	1	237	
for	1	205	
com	1	153	


In [55]:
# part e - view the top 10 records from each partition (RUN THIS CELL AS IS)
for idx in range(2):
    print(f"\n===== part-0000{idx}=====\n")
    !hdfs dfs -cat {HDFS_DIR}/eda-sort-output/part-0000{idx} | head


===== part-00000=====

the	0	549	
to	0	398	
ect	0	381	
and	0	277	
of	0	230	
hou	0	205	
a	0	195	
in	0	181	
for	0	169	
on	0	135	
cat: Unable to write to output stream.

===== part-00001=====

the	1	698	
to	1	566	
and	1	393	
your	1	357	
a	1	348	
you	1	345	
of	1	336	
in	1	237	
for	1	205	
com	1	153	
cat: Unable to write to output stream.


__Expected output:__
<table>
<th>part-00000:</th>
<th>part-00001:</th>
<tr><td><pre>
the	0	549	
to	0	398	
ect	0	382	
and	0	278	
of	0	230	
hou	0	206	
a	0	196	
in	0	182	
for	0	170	
on	0	135
</pre></td>
<td><pre>
the	1	698	
to	1	566	
and	1	392	
your	1	357	
a	1	347	
you	1	345	
of	1	336	
in	1	236	
for	1	204	
com	1	153
</pre></td></tr>
</table>

# Question 5: Counters and Combiners.
Tuning the number of mappers & reducers is helpful to optimize very large distributed computations. Doing so successfully requires a thorough understanding of the data size at each stage of the job. As you learned in the week3 live session, counters are an invaluable resource for understanding this kind of detail. In this question, we will take the EDA performed in Question 4 as an opportunity to illustrate some related concepts.

### Q5 Tasks:
* __a) short response:__ Read the Hadoop output from your job in Question 4a to report how many records are emitted by the mappers and how many records are received be the reducers. In the context of word counting what does this number represent practically?

* __b) code:__ Note that we wrote the reducer in question 4a such that the input and output record format is identical. This makes it easy to use the same reducer script as a combiner. In the space provided below, write the Hadoop Streaming command to re-run your job from question 4a with this combining added.

* __c) short response__: Report the number of records emitted by your mappers in part b and the number of records received by your reducers. Compare your results here to what you saw in part a. Explain.

* __d) short response__: Describe a scenario where using a combiner would _NOT_ improve the efficiency of the shuffle stage. Explain. [__`BONUS:`__ how does increasing the number of mappers affect the usefulness of a combiner?]

### Q5 Student Answers:
> __a)__ Map output records=31490 and Reduce input records=31490. It represents the number of intermediate key-value pairs (word,class count) that is transferred from the mapper to the reducer.

> __c)__ Map output records=31490 and  Reduce input records=13096. Since we used the combiners, there was a local aggregation that resulted in lesser records in the shuffle phase. Combiners recieved 31490 and emitted 13096 to the reducers.

> __d)__ Combiners can only be used on the functions that are associative and commutative. So, for scenarios like finding a mean, separate combiners cannot be used. Unlike reducers, a combiner will only get its data from single mapper. So, if the data block within a mapper is highly varied and there is not much repetition, use of combiner will not help. 

In [62]:
# part b - clear output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/eda-output

Deleted /user/root/HW2/eda-output


In [63]:
# part b - write your Hadoop streaming job here
!hadoop jar {JAR_FILE} \
  -files EnronEDA/reducer.py,EnronEDA/mapper.py \
  -mapper mapper.py \
  -combiner reducer.py \
  -reducer reducer.py \
  -input {HDFS_DIR}/enron.txt \
  -output {HDFS_DIR}/eda-output \
  -numReduceTasks 2 \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob7936049396824815305.jar tmpDir=null
19/01/22 22:20:26 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/22 22:20:26 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/22 22:20:27 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/22 22:20:27 INFO mapreduce.JobSubmitter: number of splits:2
19/01/22 22:20:27 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1547919087985_0009
19/01/22 22:20:27 INFO impl.YarnClientImpl: Submitted application application_1547919087985_0009
19/01/22 22:20:27 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1547919087985_0009/
19/01/22 22:20:27 INFO mapreduce.Job: Running job: job_1547919087985_0009
19/01/22 22:20:35 INFO mapreduce.Job: Job job_1547919087985_0009 running in uber mode : false
19/01/22 22:20:35 INFO mapreduce.Job:  map 0% reduce 0%
19/01

# Question 6: Document Classification Task Overview.
The week 2 assigned reading from Chapter 13 of _Introduction to Information Retrieval_ by Manning, Raghavan and Schutze provides a thorough introduction to the document classification task and the math behind Naive Bayes. In this question we'll use the example from Table 13.1 (reproduced below) to 'train' an unsmoothed Multinomial Naive Bayes model and classify a test document by hand.

<table>
<th>DocID</th>
<th>Class</th>
<th>Subject</th>
<th>Body</th>
<tr><td>Doc1</td><td>1</td><td></td><td>Chinese Beijing Chinese</td></tr>
<tr><td>Doc2</td><td>1</td><td></td><td>Chinese Chinese Shanghai</td></tr>
<tr><td>Doc3</td><td>1</td><td></td><td>Chinese Macao</td></tr>
<tr><td>Doc4</td><td>0</td><td></td><td>Tokyo Japan Chinese</td></tr>
</table>

### Q6 Tasks:
* __a) short response:__ Equation 13.3 in Manning, Raghavan and Shutze shows how a Multinomial Naive Bayes model classifies a document. It predicts the class, $c$, for which the estimated conditional probability of the class given the document's contents,  $\hat{P}(c|d)$, is greatest. In this equation what two pieces of information are required to calculate  $\hat{P}(c|d)$? Your answer should include both mathematical notatation and verbal explanation.


* __b) short response:__ The Enron data includes two classes of documents: `spam` and `ham` (they're actually labeled `1` and `0`). In plain English, explain what  $\hat{P}(c)$ and   $\hat{P}(t_{k} | c)$ mean in the context of this data. How will we would estimate these values from a training corpus? How many passes over the data would we need to make to retrieve this information for all classes and all words?


* __c) hand calculations:__ Above we've reproduced the document classification example from the textbook (we added an empty subject field to mimic the Enron data format). Remember that the classes in this "Chinese Example" are `1` (about China) and `0` (not about China). Calculate the class priors and the conditional probabilities for an __unsmoothed__ Multinomial Naive Bayes model trained on this data. Show the calculations that lead to your result using markdown and $\LaTeX$ in the space provided or by embedding an image of your hand written work. [`NOTE:` _Your results should NOT match those in the text -- they are training a model with +1 smoothing you are training a model without smoothing_]


* __d) hand calculations:__ Use the model you trained to classify the following test document: `Chinese Chinese Chinese Tokyo Japan`. Show the calculations that lead to your result using markdown and   $\LaTeX$ in the space provided or by embedding an image of your hand written work.


* __e) short response:__ Compare the classification you get from this unsmoothed model in `d`/`e` to the results in the textbook's "Example 1" which reflects a model with Laplace plus 1 smoothing. How does smoothing affect our inference?

### Q6 Student Answers:
> __a)__  We need the below two:

1) Prior probabilities of all the classes which can be obtained by : $P_c = \frac{N_c}{N}$, where $N_c$ is number of documents in class c, and N is the total number of documents
        
2) Conditional probability of each word(token) in the document given a class. The conditional probability of a single token/word can be obtained : 
        $\hat{P}(t|c) = \frac{T_{ct}}{\sum_{t`}T_{ct`}}$
   where $T_{ct}$ is the number of occurences of a given token 't' in class 'c'. Denominator represents sum of occurences of all the tokens in a given class 'c'
   
Once we have the above two calculated for every class, we can obtain the maximum a posteriori (argmax/MAP) to determine the correct class using the below formula

$c_{map} = argmax_{(c\epsilon{C})}\hat{P}(c)\prod_{1\le k\le n_d}\hat{P}(t_k|c)$

> __b)__ 

1)$\hat{P}(c)$ represents the prior probability of a given class based on the training data. In other words, it is the probability of a given class (spam/ham). For example, if we have 4 documents with 3 spam and 1 ham, then $\hat{P}(c=spam)=3/4$

2) $\hat{P}(t_k|c)$ represents the probability of a token/word $t_k$ occuring in a given class 'c'. 

We would estimate this from our training set doing the below:

* Count the number of documents classified per-class. This is required for prior probability
* tokenize the documents (Optionally remove stop words etc). 
* Count the number of occurences of each token on a per-class basis
* Count the number of tokens in a class (length of the class) 
* Obtain the conditional probability for each token per-class from "3" and "4"

The counting can be done in a single pass

> __c)__ There are 3 entries with class=1 and 1 entry with class=0. Total 4 documents are there in the training set.

Vocabulary = ['Chinese','Beijing','Shanghai','Macao','Tokyo','Japan']

For class = 1:

Prior probability : $P_{(c=1)} = 3/4$ 

Conditional Probabilities:

* $P(Chinese|c=1) = 5/8$
* $P(Beijing|c=1) = 1/8$
* $P(Shanghai|c=1) = 1/8$
* $P(Macao|c=1) = 1/8$
* $P(Tokyo|c=1) = 0/8 = 0$
* $P(Japan|c=1) = 0/8 = 0$


For class = 0:

Prior probability : $P_{(c=0)} = 1/4$ 

Conditional Probabilities:

* $P(Chinese|c=0) = 1/3$
* $P(Beijing|c=0) = 0/3 = 0$
* $P(Shanghai|c=0) = 0/3 = 0$
* $P(Macao|c=0) = 0/3 = 0$
* $P(Tokyo|c=0) = 1/3$
* $P(Japan|c=0) = 1/3$


> __d)__ 

Test = "Chinese Chinese Chinese Tokyo Japan"

$P(Test|class=1) = 3/4*(5/8)^3*0*0 = 0$

$P(Test|class=0) = 1/4*(1/3)^3*1/3*1/3 = ~0.001$

Taking the argmax, the classification results in c=0

> __e)__ If there is any word that does not occur for a given class, it will result in the overall probability of 0 for that class. Since our training data is very sparse, this results in a situation where most classes will get a 0 probbability without smoothing. In our example, the lack of words Tokyo/Japan in c=1 resulted in a probability of 0 even though the word "Chinese" is more frequent in c=1. When the smoothing of "add 1" is done, the result is different and it gets classified as c=1. Additional note: Even though not asked, the word "Chinese" occurs in every document so a TF-IDF tokenizer will penalize this word. So with smoothing and TF-IDF, our result might go back to c=0.


In [6]:
# part d/e - if you didn't write out your calcuations above, embed a picture of them here:
from IPython.display import Image
Image(filename="path-to-hand-calulations-image.png")

# Question 7: Naive Bayes Inference.
In the next two questions you'll write code to parallelize the Naive Bayes calculations that you performed above. We'll do this in two phases: one MapReduce job to perform training and a second MapReduce to perform inference. While in practice we'd need to train a model before we can use it to classify documents, for learning purposes we're going to develop our code in the opposite order. By first focusing on the pieces of information/format we need to perform the classification (inference) task you should find it easier to develop a solid implementation for training phase when you get to question 8 below. In both of these questions we'll continue to use the Chinese example corpus from the textbook to help us test our MapReduce code as we develop it. Below we've reproduced the corpus, test set and model in text format that matches the Enron data.

### Q7 Tasks:
* __a) short response:__ run the provided cells to create the example files and load them in to HDFS. Then take a closer look at __`NBmodel.txt`__. This text file represents a Naive Bayes model trained (with Laplace +1 smoothing) on the example corpus. What are the 'keys' and 'values' in this file? Which record means something slightly different than the rest? The value field of each record includes two numbers which will be helpful for debugging but which we don't actually need to perform inference -- what are they? [`HINT`: _This file represents the model from Example 13.1 in the textbook, if you're having trouble getting oriented try comparing our file to the numbers in that example._]


* __b) short response:__ When performing Naive Bayes in practice instead of multiplying the probabilities (as in equation 13.3) we add their logs (as in equation 13.4). Why do we choose to work with log probabilities? If we had an unsmoothed model, what potential error could arise from this transformation?


* __c) short response:__ Documents 6 and 8 in the test set include a word that did not appear in the training corpus (and as a result does not appear in the model). What should we do at inference time when we need a class conditional probability for this word?


* __d) short response:__ The goal of our MapReduce job is to stream over the test set and classify each document by peforming the calculation from equation 13.4. To do this we'll load the model file (which contains the probabilities for equation 13.4) into memory on the nodes where we do our mapping. This is called an in-memory join. Does loading a model 'state' like this depart from the functional programming principles? Explain why or why not. From a scability perspective when would this kind of memory use be justified? when would it be unwise?


* __e) code:__ Complete the code in __`NaiveBayes/classify_mapper.py`__. Read the docstring carefully to understand how this script should work and the format it should return. Run the provided unit tests to confirm that your script works as expected then write a Hadoop streaming job to classify the Chinese example test set. [`HINT 1:` _you shouldn't need a reducer for this one._ `HINT 2:` _Don't forget to add the model file to the_ `-files` _parameter in your Hadoop streaming job so that it gets shipped to the mapper nodes where it will be accessed by your script._]


* __f) short response:__ In our test example and in the Enron data set we have fairly short documents. Since these fit fine in memory on a mapper node we didn't need a reducer and could just do all of our calculations in the mapper. However with much longer documents (eg. books) we might want a higher level of parallelization -- for example we might want to process parts of a document on different nodes. In this hypothetical scenario how would our algorithm design change? What could the mappers still do? What key-value structure would they emit? What would the reducers have to do as a last step?

### Q7 Student Answers:
> __a)__ The keys are the 'word name' and 'ClassPriors'. Values are the counts (for chinese & non-chinese) and conditional probabilities. ClassPriors have the values that are different than the rest. The counts in the value field are not needed for inference. We only need conditional probabilities that were derived from those counts.

> __b)__ Probabilities can go very small, and the multiplication can lead to really smaller numbers. Log probabilities are easier because it keeps the numbers reasomable to understand and additional is an easier operation. If we had an unsmoothed model, the probability is 0. Log of 0 is undefined.

> __c)__ Since we are doing smoothing, the class conditional probability is taken care here. Add 1 smoothing will ensure there is some probability, albeit low for this word. The word that is missing in training set is 'Trade'

> __d)__ By doing in-memory join on the mapper nodes, we are having mappers do the entire classification job. This to me is deviating from the functional programming model. Moreover, if the vocabulary is very big, we are carrying this file on every mapper for computation.  

> __e)__ Complete the coding portion of this question before answering 'f'.

> __f)__ In the case of large dataset, I would assume that a given test document (doc_id) could be large enough that it is split across multiple nodes. I think the mappers can emit doc_id, word, logpHam, logpSpam. With doc_id being the primary key, the reducers can aggregate the conditional log probabilities for a given doc_id and add them along with class priors. Reducers will be able to emit the doc_id, logpHam, logpSpam, predicted_class. In other words, reducers will be able to determine the final predicted class.


Run these cells to create the example corpus and model.

In [64]:
%%writefile NaiveBayes/chineseTrain.txt
D1	1		Chinese Beijing Chinese
D2	1		Chinese Chinese Shanghai
D3	1		Chinese Macao
D4	0		Tokyo Japan Chinese

Writing NaiveBayes/chineseTrain.txt


In [65]:
%%writefile NaiveBayes/chineseTest.txt
D5	1		Chinese Chinese Chinese Tokyo Japan
D6	1		Beijing Shanghai Trade
D7	0		Japan Macao Tokyo
D8	0		Tokyo Japan Trade

Writing NaiveBayes/chineseTest.txt


In [66]:
%%writefile NBmodel.txt
beijing	0.0,1.0,0.111111111111,0.142857142857
chinese	1.0,5.0,0.222222222222,0.428571428571
tokyo	1.0,0.0,0.222222222222,0.0714285714286
shanghai	0.0,1.0,0.111111111111,0.142857142857
ClassPriors	1.0,3.0,0.25,0.75
japan	1.0,0.0,0.222222222222,0.0714285714286
macao	0.0,1.0,0.111111111111,0.142857142857

Writing NBmodel.txt


In [67]:
# load the data files into HDFS
!hdfs dfs -copyFromLocal NaiveBayes/chineseTrain.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal NaiveBayes/chineseTest.txt {HDFS_DIR}

Your work for `part e` starts here:

In [68]:
# part e - do your work in NaiveBayes/classify_mapper.py first, then run this cell.
!chmod a+x NaiveBayes/classify_mapper.py

In [69]:
# part e - unit test NaiveBayes/classify_mapper.py (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


In [70]:
# part e - clear the output directory in HDFS (RUN THIS CELL AS IS)
!hdfs dfs -rm -r {HDFS_DIR}/chinese-output

rm: `/user/root/HW2/chinese-output': No such file or directory


In [71]:
# part e - write your Hadooop streaming job here
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NBmodel.txt \
  -mapper classify_mapper.py \
  -input {HDFS_DIR}/chineseTest.txt \
  -output {HDFS_DIR}/chinese-output \
  -cmdenv PATH={PATH}

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob6293883176071394624.jar tmpDir=null
19/01/23 14:19:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/23 14:19:17 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/23 14:19:18 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/23 14:19:18 INFO mapreduce.JobSubmitter: number of splits:2
19/01/23 14:19:18 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1547919087985_0010
19/01/23 14:19:19 INFO impl.YarnClientImpl: Submitted application application_1547919087985_0010
19/01/23 14:19:19 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1547919087985_0010/
19/01/23 14:19:19 INFO mapreduce.Job: Running job: job_1547919087985_0010
19/01/23 14:19:25 INFO mapreduce.Job: Job job_1547919087985_0010 running in uber mode : false
19/01/23 14:19:25 INFO mapreduce.Job:  map 0% reduce 0%
19/01

In [72]:
# part e - retrieve test set results from HDFS (RUN THIS CELL AS IS)
!hdfs dfs -cat {HDFS_DIR}/chinese-output/part-000* > NaiveBayes/chineseResults.txt

In [73]:
# part e - take a look (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseResults.txt | column -t

d5  1  -8.90668134500626   -8.10769031284611   1
d6  1  -5.780743515794329  -4.179502370564408  1
d7  0  -6.591673732011658  -7.511706880737812  0
d8  0  -4.394449154674438  -5.565796731681498  0


<table>
<th> Expected output for the test set:</th>
<tr align=Left><td><pre>
d5	1	-8.90668134	-8.10769031	1
d6	1	-5.78074351	-4.17950237	1
d7	0	-6.59167373	-7.51170688	0
d8	0	-4.39444915	-5.56579673	0
</pre></td><tr>
</table>

# Question 8: Naive Bayes Training.
In Question 7 we used a model that we had trained by hand. Next we'll develop the code to do that same training in parallel, making it suitable for use with larger corpora (like the Enron emails). The end result of the MapReduce job you write in this question should be a model text file that looks just like the example (`NBmodel.txt`) that we created by hand above.

To refresh your memory about the training process take a look at  `6a` and `6b` where you described the pieces of information you'll need to collect in order to encode a Multinomial Naive Bayes model. We now want to retrieve those pieces of information while streaming over a corpus. The bulk of the task will be very similar to the word counting excercises you've already done but you may want to consider a slightly different key-value record structure to efficiently tally counts for each class. 

The most challenging (interesting?) design question will be how to retrieve the totals (# of documents and # of words in documents for each class). Of course, counting these numbers is easy. The hard part is the timing: you'll need to make sure you have the counts totalled up _before_ you start estimating the class conditional probabilities for each word. It would be best (i.e. most scalable) if we could find a way to do this tallying without storing the whole vocabulary in memory... Use an appropriate MapReduce design pattern to implement this efficiently! 


### Q8 Tasks:
* __a) make a plan:__  Fill in the docstrings for __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ to appropriately reflect the format that each script will input/output. [`HINT:` _the input files_ (`enronemail_1h.txt` & `chineseTrain.txt`) _have a prespecified format and your output file should match_ `NBmodel.txt` _so you really only have to decide on an internal format for Hadoop_].


* __b) implement it:__ Complete the code in __`NaiveBayes/train_mapper.py`__ and __`NaiveBayes/train_reducer.py`__ so that together they train a Multinomial Naive Bayes model __with no smoothing__. Make sure your end result is formatted correctly (see note above). Test your scripts independently and together (using `chineseTrain.txt` or test input of your own devising). When you are satisfied with your Python code design and run a Hadoop streaming command to run your job in parallel on the __chineseTrain.txt__. Confirm that your trained model matches your hand calculations from Question 5.


* __c) short response:__ We saw in Question 6 that adding Laplace +1 smoothing makes our classifications less sensitve to rare words. However implementing this technique requires access to one additional piece of information that we had not previously used in our Naive Bayes training. What is that extra piece of information? [`HINT:` see equation 13.7 in Manning, Raghavan and Schutze].


* __d) short response:__ There are three approaches that we could take to handle the extra piece of information you identified in `c`: 1) we could hard code it into our reducer (_where would we get it in the first place?_). Or 2) we could compute it inside the reducer which would require storing some information in memory (_what information?_). Or 3) we could compute it in the reducer without storing any bulky information in memory but then we'd need some postprocessing or a second MapReduce job to complete the calculation (_why?_). Breifly explain what is non-ideal about each of these options. BONUS: which of these options is incompatible with using multiple reducers?


* __e) code + short response:__ Choose one of the 3 options above. State your choice & reasoning in the space below then use that strategy to complete the code in __`NaiveBayes/train_reducer_smooth.py`__. Test this alternate reducer then write and run a Hadoop streaming job to train an MNB model with smoothing on the Chinese example. Your results should match the model that we provided for you above (and the calculations in the textbook example). [`HINT:` _don't start from scratch with this one -- you can just copy over your reducer code from part `b` and make the needed modifications_].

### Q8 Student Answers:
> __ c)__ We need the vocabulary size which we did not account for in the previous question.

> __ d)__ If we hard-code it in our reducer, we need to have a prior knowledge of vocabulary words. In large corpus, I think we can just take standard vocabulary from GLOVE or Word2Vec with fixed size (say 60K) for hard coding. For option 2 and 3, we need to count the exact unique words from our training data. Calculating the vocab size dynamically need to be done at the reducer. Option 2 will break if we have multiple reducers, since we will not get all the words in a single reducer. Option 3 is the best in my opinion, which involves pre-processing of information or another map-reduce task. 

> __ e)__ I wrote code to calculate the vocab size offline (as part of EDA). I am passing this as an argument to the reducer in the stream job. I have written the code to dynamically check the number of reducers that are passed. Since the number of reducers can be more than one, I could not go for in-memory vocab size calculation in the reducer. 


In [5]:
# part a - do your work in train_mapper.py and train_reducer.py then RUN THIS CELL AS IS
!chmod a+x NaiveBayes/train_mapper.py
!chmod a+x NaiveBayes/train_reducer.py
!echo "=========== MAPPER DOCSTRING ============"
!head -n 8 NaiveBayes/train_mapper.py | tail -n 6
!echo "=========== REDUCER DOCSTRING ============"
!head -n 12 NaiveBayes/train_reducer.py | tail -n 12

=========== MAPPER DOCSTRING ============
Mapper reads in text documents and emits word counts by class.
   
INPUT:
    ID \t class \t subject \t body 
OUTPUT:
    pkey \t word \t count \t class
=========== REDUCER DOCSTRING ============
#!/usr/bin/env python
"""
Reducer aggregates word counts by class and emits frequencies.

INPUT:
    pkey \t word \t count \t class
OUTPUT:
    For words:
    word \t class_0_count \t class_1_count \t conditional_class_0 \t conditional_class_1
    For class prior:
    ClassPrior \t class_0_count \t class_1_count \t probability_class_0 \t probability_class_1
    


__`part b starts here`:__ MNB _without_ Smoothing (training on Chinese Example Corpus).

In [181]:
# part b - write a unit test for your mapper here
!echo -e "d1	1	cricket	bat ball wickets gloves\nd2	0	football	ball helmet gloves" | NaiveBayes/train_mapper.py

A	cricket	1	1
A	bat	1	1
A	ball	1	1
A	wickets	1	1
A	gloves	1	1
A	football	1	0
A	ball	1	0
A	helmet	1	0
A	gloves	1	0
A	*class0_w_count	4	0
A	*class1_w_count	5	1
A	*class0_count	1	0
A	*class1_count	1	1


In [182]:
# part b - write a unit test for your reducer here
!echo -e "A	ball	1	0\nA	ball	1	1\nA	*class0_w_count	4	0\nA	*class1_w_count	5	1\nA	*class0_count	1	0\nA	*class1_count	1	1" | sort -k2 | NaiveBayes/train_reducer.py

ball	1,1,0.25,0.2
ClassPriors	1.0,1.0,0.5,0.5


In [183]:
# part b - write a systems test for your mapper + reducer together here
!echo -e "d1	1	cricket	bat ball wickets gloves\nd2	0	football	ball helmet gloves" | NaiveBayes/train_mapper.py | sort -k2 | NaiveBayes/train_reducer.py

ball	1,1,0.25,0.2
bat	0,1,0.0,0.2
cricket	0,1,0.0,0.2
football	1,0,0.25,0.0
gloves	1,1,0.25,0.2
helmet	1,0,0.25,0.0
wickets	0,1,0.0,0.2
ClassPriors	1.0,1.0,0.5,0.5


In [184]:
!cat NaiveBayes/ChineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2 | NaiveBayes/train_reducer.py

beijing	0,1,0.0,0.125
chinese	1,5,0.3333333333333333,0.625
japan	1,0,0.3333333333333333,0.0
macao	0,1,0.0,0.125
shanghai	0,1,0.0,0.125
tokyo	1,0,0.3333333333333333,0.0
ClassPriors	1.0,3.0,0.25,0.75


In [135]:
# part b - clear (and name) an output directory in HDFS for your unsmoothed chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/nb-chinese-output

Deleted /user/root/HW2/nb-chinese-output


In [136]:
# part b - write your hadoop streaming job
# We use 4 reducers for this streaming job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/nb-chinese-output \
  -cmdenv PATH={PATH}\
  -numReduceTasks 4

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob4039302219449088383.jar tmpDir=null
19/01/28 19:18:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/28 19:18:57 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/28 19:18:58 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/28 19:18:58 INFO mapreduce.JobSubmitter: number of splits:2
19/01/28 19:18:58 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1548343727265_0066
19/01/28 19:18:59 INFO impl.YarnClientImpl: Submitted application application_1548343727265_0066
19/01/28 19:18:59 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1548343727265_0066/
19/01/28 19:18:59 INFO mapreduce.Job: Running job: job_1548343727265_0066
19/01/28 19:19:07 INFO mapreduce.Job: Job job_1548343727265_0066 running in uber mode : false
19/01/28 19:19:07 INFO mapreduce.Job:  map 0% reduce 0%
19/01

In [137]:
for p in range(4):
    print('='*10,f'PARTITION{p+1}','='*10)
    !hdfs dfs -cat {HDFS_DIR}/nb-chinese-output/part-0000{p} | head -n 15

========== PARTITION1 ==========
shanghai	0,1,0.0,0.125
tokyo	1,0,0.3333333333333333,0.0
========== PARTITION2 ==========
beijing	0,1,0.0,0.125
chinese	1,5,0.3333333333333333,0.625
ClassPriors	1.0,3.0,0.25,0.75
========== PARTITION3 ==========
japan	1,0,0.3333333333333333,0.0
========== PARTITION4 ==========
macao	0,1,0.0,0.125


In [138]:
# part b - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/nb-chinese-output/part-0000* > NaiveBayes/NBmodel_no_smooth.txt

In [139]:
# part b - print your model so that we can confirm that it matches expected results
!cat NaiveBayes/NBmodel_no_smooth.txt

shanghai	0,1,0.0,0.125
tokyo	1,0,0.3333333333333333,0.0
beijing	0,1,0.0,0.125
chinese	1,5,0.3333333333333333,0.625
ClassPriors	1.0,3.0,0.25,0.75
japan	1,0,0.3333333333333333,0.0
macao	0,1,0.0,0.125


__`part e starts here`:__ MNB _with_ Smoothing (training on Chinese Example Corpus).

In [140]:
!chmod 777 NaiveBayes/train_reducer_smooth.py

In [186]:
# part e - write a unit test for your NEW reducer here
!echo -e "A	ball	1	0\nA	ball	1	1\nA	*class0_w_count	4	0\nA	*class1_w_count	5	1\nA	*class0_count	1	0\nA	*class1_count	1	1" | sort -k2 | NaiveBayes/train_reducer_smooth.py 6

ball	1,1,0.2,0.18181818181818182
ClassPriors	1.0,1.0,0.5,0.5


In [190]:
# To get the vocab size, wrote a script that we will use.
!cat NaiveBayes/ChineseTrain.txt | ./vocab_size.py

Length of the vocabulary:  6


In [187]:
# part e - write a systems test for your mapper + reducer together here
# We pass vocab_size = 6 as argument for unit test
!cat NaiveBayes/ChineseTrain.txt | NaiveBayes/train_mapper.py | sort -k2 | NaiveBayes/train_reducer_smooth.py 6

beijing	0,1,0.1111111111111111,0.14285714285714285
chinese	1,5,0.2222222222222222,0.42857142857142855
japan	1,0,0.2222222222222222,0.07142857142857142
macao	0,1,0.1111111111111111,0.14285714285714285
shanghai	0,1,0.1111111111111111,0.14285714285714285
tokyo	1,0,0.2222222222222222,0.07142857142857142
ClassPriors	1.0,3.0,0.25,0.75


In [195]:
# part e - clear (and name) an output directory in HDFS for your SMOOTHED chinese NB model
!hdfs dfs -rm -r {HDFS_DIR}/nb-chinese-smooth-output

Deleted /user/root/HW2/nb-chinese-smooth-output


In [196]:
# part e - write your hadoop streaming job
# We use 1 reducer to check our code. Verified with 1 and 4 reducers.
# We also pass the vocab_size as an argument. 
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper train_mapper.py \
  -reducer "train_reducer_smooth.py 6" \
  -input {HDFS_DIR}/chineseTrain.txt \
  -output {HDFS_DIR}/nb-chinese-smooth-output \
  -cmdenv PATH={PATH}\
  -numReduceTasks 1

packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8164764589634224967.jar tmpDir=null
19/01/31 00:33:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/31 00:33:46 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/31 00:33:47 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/31 00:33:47 INFO mapreduce.JobSubmitter: number of splits:2
19/01/31 00:33:47 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1548343727265_0088
19/01/31 00:33:48 INFO impl.YarnClientImpl: Submitted application application_1548343727265_0088
19/01/31 00:33:48 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1548343727265_0088/
19/01/31 00:33:48 INFO mapreduce.Job: Running job: job_1548343727265_0088
19/01/31 00:33:56 INFO mapreduce.Job: Job job_1548343727265_0088 running in uber mode : false
19/01/31 00:33:56 INFO mapreduce.Job:  map 0% reduce 0%
19/01

In [197]:
# part e - extract your results (i.e. model) to a local file
!hdfs dfs -cat {HDFS_DIR}/nb-chinese-smooth-output/part-0000* > NaiveBayes/NBmodel_smooth.txt

In [198]:
# part 2 - Display the model
!cat NaiveBayes/NBmodel_smooth.txt

beijing	0,1,0.1111111111111111,0.14285714285714285
chinese	1,5,0.2222222222222222,0.42857142857142855
japan	1,0,0.2222222222222222,0.07142857142857142
macao	0,1,0.1111111111111111,0.14285714285714285
shanghai	0,1,0.1111111111111111,0.14285714285714285
tokyo	1,0,0.2222222222222222,0.07142857142857142
ClassPriors	1.0,3.0,0.25,0.75


# Question 9: Enron Ham/Spam NB Classifier & Results.

Fantastic work. We're finally ready to perform Spam Classification on the Enron Corpus. In this question you'll run the analysis you've developed, report its performance, and draw some conclusions.

### Q9 Tasks:
* __a) train/test split:__ Run the provided code to split our Enron file into a training set and testing set then load them into HDFS. 

[`NOTE:` _If you hard coded the vocab size in question 8d make sure you re calculate the vocab size for just the training set!_]

* __b) train 2 models:__ Write Hadoop Streaming jobs to train MNB Models on the training set with and without smoothing. Save your models to local files at __`NaiveBayes/Unsmoothed/NBmodel.txt`__ and __`NaiveBayes/Smoothed/NBmodel.txt`__. [`NOTE:` _This naming is important because we wrote our classification task so that it expects a file of that name... if this inelegance frustrates you there is an alternative that would involve a few adjustments to your code [read more about it here](http://www.tnoda.com/blog/2013-11-23)._] Finally run the checks that we provide to confirm that your results are correct.


* __c) code:__ Recall that we designed our classification job with just a mapper. An efficient way to report the performance of our models would be to simply add a reducer phase to this job and compute precision and recall right there. Complete the code in __`NaiveBayes/evaluation_reducer.py`__ and then write Hadoop jobs to evaluate your two models on the test set. Report their performance side by side. [`NOTE:` if you need a refresher on precision, recall and F1-score [Wikipedia](https://en.wikipedia.org/wiki/F1_score) is a good resource.]


* __d) short response:__ Compare the performance of your two models. What do you notice about the unsmoothed model's predictions? Can you guess why this is happening? Which evaluation measure do you think is most relevant in our use case? [`NOTE:` _Feel free to answer using your common sense but if you want more information on evaluating the classification task checkout_ [this blogpost](https://tryolabs.com/blog/2013/03/25/why-accuracy-alone-bad-measure-classification-tasks-and-what-we-can-do-about-it/
) or [this paper](http://www.flinders.edu.au/science_engineering/fms/School-CSEM/publications/tech_reps-research_artfcts/TRRA_2007.pdf
)]


* __e) code + short response:__ Let's look at the top ten words with the highest conditional probability in `Spam` and in `Ham`. We'll do this by writing a Hadoop job that sorts the model file (`NaiveBayes/Smoothed/NBmodel.py`). Normally we'd have to run two jobs -- one that sorts on $P(word|ham)$ and another that sorts on $P(word|spam)$. However if we slighly modify the data format in the model file then we can get the top words in each class with just one job. We've written a mapper that will do just this for you. Read through __`NaiveBayes/model_sort_mapper.py`__ and then briefly explain how this mapper will allow us to partition and sort our model file. Write a Hadoop job that uses our mapper and `/bin/cat` for a reducer to partition and sort. Print out the top 10 words in each class (where 'top' == highest conditional probability).[`HINT:` _this should remind you a lot of what we did in Question 6._]


* __f) short response:__ What do you notice about the 'top words' we printed in `e`? How would increasing the smoothing parameter 'k' affect the probabilities for the top words that you identified for 'e'. How would they affect the probabilities of words that occur much more in one class than another? In summary, how does the smoothing parameter 'k' affect the bias and the variance of our model. [`NOTE:` _you do not need to code anything for this task, but if you are struggling with it you could try changing 'k' and see what happens to the test set. We don't recommend doing this exploration with the Enron data because it will be harder to see the impact with such a big vocabulary_]

### Q9 Student Answers:
> __d)__ The unsmoothed model gives pretty bad metrics because of the new words appearing in the test set that do not exist in the training set. The most relevant metrics for our use case are recall and F-Score. Precision is a bad choice, because it only depends on the TP & FP scores. Because of the missing values, the model did not predict too many positives (only 1 TP and 0 FP), thus making the precision very good. 

> __e)__ The mapper model_sort_mapper.py is adding two additional columns: 'ham/spam' and 'conditional probability'. This allows us to first sort based on ham/spam prediction and then secondary sort on the probability. 

> __f)__ I notice that the top outputs of ham and spam are very common words with very similar conditional probabilities. Increasing 'K' will have minimal impact on words that have huge difference in conditional probabilities of ham and spam. If we look at classPriors, it has more ham than spam. So, increasing K will move conditional probabilities of words more towards spam than to ham. So, if there is a word that is declared as 'ham' but the delta between ham/spam is very less (as we saw with common words), increasing the K might change their max probability towards spam. For the words like 'enron' and 'hou' which occur much more in ham than in spam, increasing K might not change the final outcome.

__Test/Train split__

In [80]:
# part a - test/train split (RUN THIS CELL AS IS)
!head -n 80 data/enronemail_1h.txt > data/enron_train.txt
!tail -n 20 data/enronemail_1h.txt > data/enron_test.txt
!hdfs dfs -copyFromLocal data/enron_train.txt {HDFS_DIR}
!hdfs dfs -copyFromLocal data/enron_test.txt {HDFS_DIR}

__Training__ (Enron MNB Model _without smoothing_ )

In [149]:
# part b -  Unsmoothed model (FILL IN THE MISSING CODE BELOW)
# We use 4 reducers for this job

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-model

# hadoop command
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer.py \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper train_mapper.py \
  -reducer train_reducer.py \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/enron-model \
  -cmdenv PATH={PATH}\
  -numReduceTasks 4

# save the model locally
!mkdir NaiveBayes/Unsmoothed
!hdfs dfs -cat {HDFS_DIR}/enron-model/part-000* > NaiveBayes/Unsmoothed/NBmodel.txt

Deleted /user/root/HW2/enron-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8576373848526817434.jar tmpDir=null
19/01/28 19:38:12 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/28 19:38:12 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/28 19:38:13 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/28 19:38:13 INFO mapreduce.JobSubmitter: number of splits:2
19/01/28 19:38:13 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1548343727265_0068
19/01/28 19:38:13 INFO impl.YarnClientImpl: Submitted application application_1548343727265_0068
19/01/28 19:38:14 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1548343727265_0068/
19/01/28 19:38:14 INFO mapreduce.Job: Running job: job_1548343727265_0068
19/01/28 19:38:23 INFO mapreduce.Job: Job job_1548343727265_0068 running in uber mode : false
19/01/28 19:38:23 INFO map

In [150]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000172547666293,0.000296823983378

assistance	2,4,0.0001725476662928134,0.00029682398337785694


In [151]:
# part b - check your UNSMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Unsmoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,8.62738331464e-05,0.00163253190858

money	1,22,8.62738331464067e-05,0.001632531908578213


__Training__ (Enron MNB Model _with Laplace +1 smoothing_ )

In [200]:
# Find the vocab size offline
!cat data/enron_train.txt | ./vocab_size.py

Length of the vocabulary:  4555


In [202]:
# part b -  Smoothed model (FILL IN THE MISSING CODE BELOW)

# clear the output directory
!hdfs dfs -rm -r {HDFS_DIR}/smooth-model

# hadoop command
# We pass vocab_size 4555 that we obtained from before
# Trying this with 4 reducers 

!hadoop jar {JAR_FILE} \
  -files NaiveBayes/train_mapper.py,NaiveBayes/train_reducer_smooth.py \
  -D stream.num.map.output.key.fields=2 \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k2,2" \
  -D mapreduce.partition.keypartitioner.options="-k1,1" \
  -partitioner org.apache.hadoop.mapred.lib.KeyFieldBasedPartitioner \
  -mapper train_mapper.py \
  -reducer "train_reducer_smooth.py 4555" \
  -input {HDFS_DIR}/enron_train.txt \
  -output {HDFS_DIR}/smooth-model \
  -cmdenv PATH={PATH}\
  -numReduceTasks 4

# save the model locally
!mkdir NaiveBayes/Smoothed
!hdfs dfs -cat {HDFS_DIR}/smooth-model/part-000* > NaiveBayes/Smoothed/NBmodel.txt

Deleted /user/root/HW2/smooth-model
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob8196994666214151966.jar tmpDir=null
19/01/31 00:39:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/31 00:39:53 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/31 00:39:54 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/31 00:39:54 INFO mapreduce.JobSubmitter: number of splits:2
19/01/31 00:39:55 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1548343727265_0090
19/01/31 00:39:55 INFO impl.YarnClientImpl: Submitted application application_1548343727265_0090
19/01/31 00:39:55 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1548343727265_0090/
19/01/31 00:39:55 INFO mapreduce.Job: Running job: job_1548343727265_0090
19/01/31 00:40:04 INFO mapreduce.Job: Job job_1548343727265_0090 running in uber mode : false
19/01/31 00:40:04 INFO ma

In [203]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep assistance NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: assistance	2,4,0.000185804533631,0.000277300205202

assistance	2,4,0.0001858045336306206,0.00027730020520215184


In [204]:
# part b - check your SMOOTHED model results (RUN THIS CELL AS IS)
!grep money NaiveBayes/Smoothed/NBmodel.txt
# EXPECTED OUTPUT: money	1,22,0.000123869689087,0.00127558094393

money	1,22,0.0001238696890870804,0.0012755809439298986


__Evaluation__

In [205]:
# part c - write your code in NaiveBayes/evaluation_reducer.py then RUN THIS
!chmod a+x NaiveBayes/evaluation_reducer.py

In [206]:
# part c - unit test your evaluation job on the chinese model (RUN THIS CELL AS IS)
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py 
!cat NaiveBayes/chineseTest.txt | NaiveBayes/classify_mapper.py | NaiveBayes/evaluation_reducer.py

d5	1	-8.90668134500626	-8.10769031284611	1
d6	1	-5.780743515794329	-4.179502370564408	1
d7	0	-6.591673732011658	-7.511706880737812	0
d8	0	-4.394449154674438	-5.565796731681498	0
d5	1	-8.90668134500626	-8.10769031284611	 True
d6	1	-5.780743515794329	-4.179502370564408	 True
d7	0	-6.591673732011658	-7.511706880737812	 True
d8	0	-4.394449154674438	-5.565796731681498	 True

FalsePositive:0.0
FalseNegative:0.0
TruePositive:2.0
TrueNegative:2.0

Precision:1.0
Recall:1.0
Accuracy:1.0
F-score:1.0


In [207]:
# part c - Evaluate the UNSMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-unsmooth-eval

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Unsmoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/enron-unsmooth-eval \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/enron-unsmooth-eval/part-000* > NaiveBayes/Unsmoothed/results.txt

Deleted /user/root/HW2/enron-unsmooth-eval
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob2113112495552157100.jar tmpDir=null
19/01/31 00:41:31 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/31 00:41:32 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/31 00:41:33 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/31 00:41:33 INFO mapreduce.JobSubmitter: number of splits:2
19/01/31 00:41:33 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1548343727265_0091
19/01/31 00:41:33 INFO impl.YarnClientImpl: Submitted application application_1548343727265_0091
19/01/31 00:41:33 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1548343727265_0091/
19/01/31 00:41:33 INFO mapreduce.Job: Running job: job_1548343727265_0091
19/01/31 00:41:43 INFO mapreduce.Job: Job job_1548343727265_0091 running in uber mode : false
19/01/31 00:41:43 

In [208]:
# part c - Evaluate the SMOOTHED Model Here (FILL IN THE MISSING CODE)

# clear output directory
!hdfs dfs -rm -r {HDFS_DIR}/enron-smooth-eval

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/classify_mapper.py,NaiveBayes/evaluation_reducer.py,NaiveBayes/Smoothed/NBmodel.txt \
  -mapper classify_mapper.py \
  -reducer evaluation_reducer.py \
  -input {HDFS_DIR}/enron_test.txt \
  -output {HDFS_DIR}/enron-smooth-eval \
  -cmdenv PATH={PATH}

# retrieve results locally
!hdfs dfs -cat {HDFS_DIR}/enron-smooth-eval/part-000* > NaiveBayes/Smoothed/results.txt

Deleted /user/root/HW2/enron-smooth-eval
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob144630576799490873.jar tmpDir=null
19/01/31 00:42:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/31 00:42:14 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/31 00:42:16 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/31 00:42:16 INFO mapreduce.JobSubmitter: number of splits:2
19/01/31 00:42:16 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1548343727265_0092
19/01/31 00:42:16 INFO impl.YarnClientImpl: Submitted application application_1548343727265_0092
19/01/31 00:42:16 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1548343727265_0092/
19/01/31 00:42:16 INFO mapreduce.Job: Running job: job_1548343727265_0092
19/01/31 00:42:26 INFO mapreduce.Job: Job job_1548343727265_0092 running in uber mode : false
19/01/31 00:42:26 INF

In [209]:
# part c - display results 
# NOTE: feel free to modify the tail commands to match the format of your results file
print('=========== UNSMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Unsmoothed/results.txt
print('=========== SMOOTHED MODEL ============')
!tail -n 9 NaiveBayes/Smoothed/results.txt

=========== UNSMOOTHED MODEL ============
FalsePositive:0.0	
FalseNegative:10.0	
TruePositive:1.0	
TrueNegative:9.0	
	
Precision:1.0	
Recall:0.09090909090909091	
Accuracy:0.5	
F-score:0.16666666666666669	
=========== SMOOTHED MODEL ============
FalsePositive:3.0	
FalseNegative:0.0	
TruePositive:11.0	
TrueNegative:6.0	
	
Precision:0.7857142857142857	
Recall:1.0	
Accuracy:0.85	
F-score:0.88	


__`EXPECTED RESULTS:`__ 
<table>
<th>Unsmoothed Model</th>
<th>Smoothed Model</th>
<tr>
<td><pre>
# Documents:	20
True Positives:	1
True Negatives:	9
False Positives:	0
False Negatives:	10
Accuracy	0.5
Precision	1.0
Recall	0.0909
F-Score	0.1666
</pre></td>
<td><pre>
# Documents:	20
True Positives:	11
True Negatives:	6
False Positives:	3
False Negatives:	0
Accuracy	0.85
Precision	0.7857
Recall	1.0
F-Score	0.88
</pre></td>
</tr>
</table>

__`NOTE:`__ _Don't be too disappointed if these seem low to you. We've trained and tested on a very very small corpus... bigger datasets coming soon!_

__`part e starts here:`__

In [111]:
# part e - write your Hadoop job here (sort smoothed model on P(word|class))

!hdfs dfs -copyFromLocal NaiveBayes/Smoothed/NBmodel.txt {HDFS_DIR}

# clear output directory

!hdfs dfs -rm -r {HDFS_DIR}/enron-smooth-sorted

# hadoop job
!hadoop jar {JAR_FILE} \
  -files NaiveBayes/model_sort_mapper.py \
  -D stream.num.map.output.key.fields=4 \
  -D stream.map.output.field.separator="\t" \
  -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapred.lib.KeyFieldBasedComparator \
  -D mapreduce.partition.keycomparator.options="-k3,3 -k4,4nr" \
  -mapper model_sort_mapper.py \
  -reducer /bin/cat \
  -input {HDFS_DIR}/NBmodel.txt \
  -output {HDFS_DIR}/enron-smooth-sorted \
  -cmdenv PATH={PATH}

# Store the results locally
!hdfs dfs -cat {HDFS_DIR}/enron-smooth-sorted/part-000* > NaiveBayes/Smoothed/enron-model-sorted.txt


rm: `/user/root/HW2/enron-smooth-sorted': No such file or directory
packageJobJar: [] [/usr/lib/hadoop-mapreduce/hadoop-streaming-2.6.0-cdh5.15.1.jar] /tmp/streamjob9164079604629233139.jar tmpDir=null
19/01/28 00:50:52 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/28 00:50:52 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
19/01/28 00:50:53 INFO mapred.FileInputFormat: Total input paths to process : 1
19/01/28 00:50:53 INFO mapreduce.JobSubmitter: number of splits:2
19/01/28 00:50:54 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1548343727265_0059
19/01/28 00:50:54 INFO impl.YarnClientImpl: Submitted application application_1548343727265_0059
19/01/28 00:50:54 INFO mapreduce.Job: The url to track the job: http://docker.w261:8088/proxy/application_1548343727265_0059/
19/01/28 00:50:54 INFO mapreduce.Job: Running job: job_1548343727265_0059
19/01/28 00:51:03 INFO mapreduce.Job: Job job_1548343727265_0059 running in uber mode :

In [112]:
# part e - print top words in each class
!head -10 NaiveBayes/Smoothed/enron-model-sorted.txt

ClassPriors	47.0,33.0,0.5875,0.4125	ham	0.5875	
ect	378,0,0.023473306082001735,5.546004104043037e-05	ham	0.023473306082001735	
and	258,277,0.01604112473677691,0.015417891409239643	ham	0.01604112473677691	
hou	203,0,0.0126347082868822,5.546004104043037e-05	ham	0.0126347082868822	
in	160,157,0.009971509971509971,0.008762686484387999	ham	0.009971509971509971	
for	148,153,0.00922829183698749,0.008540846320226277	ham	0.00922829183698749	
on	122,95,0.007617985878855444,0.005324163939881316	ham	0.007617985878855444	
enron	116,0,0.007246376811594203,5.546004104043037e-05	ham	0.007246376811594203	
will	113,69,0.007060572277963582,0.003882202872830126	ham	0.007060572277963582	
i	113,106,0.007060572277963582,0.00593422439132605	ham	0.007060572277963582	


In [113]:
# Spam starts from line 1955. Display top 10 spam
!awk 'NR >= 1955 && NR <= 1964' NaiveBayes/Smoothed/enron-model-sorted.txt

the	453,535,0.02811841942276725,0.029726581997670677	spam	0.029726581997670677	
to	350,420,0.021739130434782608,0.023348677278021184	spam	0.023348677278021184	
a	168,274,0.010466988727858293,0.015251511286118352	spam	0.015251511286118352	
your	35,271,0.002229654403567447,0.01508513116299706	spam	0.01508513116299706	
you	80,252,0.005016722408026756,0.014031390383228884	spam	0.014031390383228884	
of	188,252,0.011705685618729096,0.014031390383228884	spam	0.014031390383228884	
it	30,119,0.0019199801808497462,0.0066552049248516446	spam	0.0066552049248516446	
com	74,108,0.004645113340765515,0.006045144473406911	spam	0.006045144473406911	
that	71,100,0.004459308807134894,0.005601464145083467	spam	0.005601464145083467	
or	41,88,0.002601263470828688,0.004935943652598303	spam	0.004935943652598303	


# HW2 ends here, please refer to the `README.md` for submission instructions.